In [1]:
import os
cwd = os.getcwd()

In [2]:
cd {cwd}

/mnt/File/shamgholi/projects/person_reid/AP3D


# load model

In [3]:
import sys
import os
sys.path.append(cwd)
import time
import math
import os.path as osp
import numpy as np

import torch
from torch.utils.data import DataLoader

from config import Config
import models
import transforms.spatial_transforms as ST
import transforms.temporal_transforms as TT
import tools.data_manager as data_manager
from tools.video_loader import VideoDataset
from tools.utils import Logger
from tools.eval_metrics import evaluate
from commons import modify_model
from tqdm import tqdm
import random


def seed_everythings():
    seed = 1
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)



class Args:
    #"/home/hadi/iust/datasets"
    #"/home/shamgholi/iust/datasets" 
    #"/mnt/File/shamgholi/datasets"
    root = "/mnt/File/shamgholi/datasets"
    dataset = "mars"
    workers = 0
    height = 256
    width = 128
    # test_frames = 8
    arch = "ap3dres50"
    resume = "./"
    pretrain = os.path.join(cwd, "logs/row41/best_model.pth.tar")
    test_epochs = [240]
    distance = "cosine" #cosine
    gpu = "0"
    test_batch = 6
    

# Default Test (Eval)

In [4]:
def test(model, queryloader, galleryloader, use_gpu, args):
    # test using 4 frames
    since = time.time()
    model.eval()

    qf, q_pids, q_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(queryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        qf.append(feat)
        q_pids.extend(pids)
        q_camids.extend(camids)

    qf = torch.cat(qf, 0)
    q_pids = np.asarray(q_pids)
    q_camids = np.asarray(q_camids)
    print("Extracted features for query set, obtained {} matrix".format(qf.shape))

    gf, g_pids, g_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(galleryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        gf.append(feat)
        g_pids.extend(pids)
        g_camids.extend(camids)

    gf = torch.cat(gf, 0)
    g_pids = np.asarray(g_pids)
    g_camids = np.asarray(g_camids)

    if args.dataset == 'mars':
        # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
        gf = torch.cat((qf, gf), 0)
        g_pids = np.append(q_pids, g_pids)
        g_camids = np.append(q_camids, g_camids)

    print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

    time_elapsed = time.time() - since
    print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print("Computing distance matrix")
    m, n = qf.size(0), gf.size(0)
    distmat = torch.zeros((m,n))

    if args.distance == 'euclidean':
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        for i in range(m):
            distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
    else:
        q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
        g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
        qf = qf.div(q_norm.expand_as(qf))
        gf = gf.div(g_norm.expand_as(gf))
        for i in range(m):
            distmat[i] = - torch.mm(qf[i:i+1], gf.t())
    distmat = distmat.numpy()
    
    print("Computing CMC and mAP")
    cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

    print("Results ----------")
    print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
    print("------------------")

    return cmc[0]




def experiment():
    print("Initializing model: {}".format(args.arch))
    seed_everythings()
    model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
    print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
    model = model.to(device)
    model = model.eval()
    modify_model(model, args, conf)
    with torch.no_grad():
        test(model, queryloader, galleryloader, use_gpu, args)
        # mytest(model, queryloader, galleryloader, use_gpu, args)


# My Test (Eval)

In [5]:
def mytest(model, queryloader, galleryloader, use_gpu, args):  
    # test using 4 frames
    since = time.time()
    model.eval()

    qf, q_pids, q_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(queryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        # feat = model.bn(feat)
        feat = feat.data.cpu()

        qf.append(feat)
        q_pids.extend(pids)
        q_camids.extend(camids)

    qf = torch.cat(qf, 0)
    q_pids = np.asarray(q_pids)
    q_camids = np.asarray(q_camids)
    print("Extracted features for query set, obtained {} matrix".format(qf.shape))

    gf, g_pids, g_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(galleryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        # feat = model.bn(feat)
        feat = feat.data.cpu()

        gf.append(feat)
        g_pids.extend(pids)
        g_camids.extend(camids)

    gf = torch.cat(gf, 0)
    g_pids = np.asarray(g_pids)
    g_camids = np.asarray(g_camids)

    if args.dataset == 'mars':
        # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
        gf = torch.cat((qf, gf), 0)
        g_pids = np.append(q_pids, g_pids)
        g_camids = np.append(q_camids, g_camids)

    print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

    time_elapsed = time.time() - since
    print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print("Computing distance matrix")
    m, n = qf.size(0), gf.size(0)
    distmat = torch.zeros((m,n))

    if args.distance == 'euclidean':
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        for i in range(m):
            distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
    else:
        q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
        g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
        qf = qf.div(q_norm.expand_as(qf))
        gf = gf.div(g_norm.expand_as(gf))
        for i in range(m):
            distmat[i] = - torch.mm(qf[i:i+1], gf.t())
    distmat = distmat.numpy()
    
    print("Computing CMC and mAP")
    cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

    print("Results ----------")
    print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
    print("------------------")

    return cmc[0]


def myexperiment():
    print("Initializing model: {}".format(args.arch))
    seed_everythings()
    model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
    print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
    model = model.to(device)
    model = model.eval()
    modify_model(model, args, conf)
    with torch.no_grad():
        # test(model, queryloader, galleryloader, use_gpu, args)
        mytest(model, queryloader, galleryloader, use_gpu, args)


# prepare date

In [15]:


# Data augmentation
args = Args()
conf = Config()
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
pin_memory = True if use_gpu else False
temporal_transform_test = TT.TemporalBeginCrop()
dataset = data_manager.init_dataset(name=args.dataset, root=args.root)
spatial_transform_test = ST.Compose([
            ST.Scale((args.height, args.width), interpolation=3),
            ST.ToTensor(),
            ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
queryloader = DataLoader(
    VideoDataset(dataset.query, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)
galleryloader = DataLoader(
    VideoDataset(dataset.gallery, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)

# return queryloader, galleryloader


=> MARS loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   625 |     8298
  query    |   626 |     1980
  gallery  |   622 |     9330
  ------------------------------
  total    |  1247 |    19608
  number of images per tracklet: 2 ~ 920, average 59.5
  ------------------------------


# Experiments

In [7]:
# conf.use_linear_to_get_important_features = False
#conf.print_model_parameters_trainable = False
# conf.use_linear_to_merge_features = False
# conf.use_hist = False
# args.pretrain = 'logs/row41/best_model.pth.tar'
# conf.print_model_layers = False
# myexperiment()
##top1:79.3% top5:91.6% top10:94.0% mAP:69.8%

In [16]:
%%time
conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = True
conf.concat_hist_max = False
conf.init_hist("HistYusufLayer")
conf.print_model_layers = False
args.pretrain = 'logs/row41/best_model.pth.tar'
seed_everythings()
model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
# print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
model = model.to(device)
model = model.eval()
modify_model(model, args, conf)
with torch.no_grad():
    # test(model, queryloader, galleryloader, use_gpu, args)
    mytest(model, queryloader, galleryloader, use_gpu, args)


Model size: 14.66900M
--------------------
RUNTIMEERROR IN LOADING BATCHNORM STATEDICT, WEIGHTS OF BN IS NOW RANDOM
--------------------
pretrain state dict loaded
----------
Model size: 14.66900M
Extracted features for query set, obtained torch.Size([1980, 5120]) matrix
Extracted features for gallery set, obtained torch.Size([11310, 5120]) matrix
Extracting features complete in 1m 43s
Computing distance matrix
Computing CMC and mAP
Results ----------
top1:62.6% top5:77.2% top10:82.4% mAP:41.9%
------------------
CPU times: user 10min 47s, sys: 40.6 s, total: 11min 27s
Wall time: 1min 57s


In [18]:
temporal_transform_test = TT.TemporalBeginCrop()
args.dataset = "duke"
dataset = data_manager.init_dataset(name=args.dataset, root=args.root)
spatial_transform_test = ST.Compose([
            ST.Scale((args.height, args.width), interpolation=3),
            ST.ToTensor(),
            ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
queryloader = DataLoader(
    VideoDataset(dataset.query, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)
galleryloader = DataLoader(
    VideoDataset(dataset.gallery, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)


Note: if root path is changed, the previously generated json files need to be re-generated (so delete them first)
=> Automatically generating split (might take a while for the first time, have a coffe)
Processing /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/train with 702 person identities
Saving split to /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/split_train.json
=> Automatically generating split (might take a while for the first time, have a coffe)
Processing /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/train with 702 person identities
Saving split to /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/split_train_dense.json
=> Automatically generating split (might take a while for the first time, have a coffe)
Processing /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/query with 702 person identities
Saving split to /mnt/File/shamgholi/datasets/DukeMTMC-VideoReID/split_query.json
=> Automatically generating split (might take a while for the first time, have a coffe)
Processin

In [20]:
conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = False
args.pretrain = 'logs/row41/best_model.pth.tar'
conf.print_model_layers = False
experiment()


Initializing model: ap3dres50
Model size: 11.79881M
pretrain state dict loaded
----------
Model size: 11.79881M
Extracted features for query set, obtained torch.Size([702, 512]) matrix
Extracted features for gallery set, obtained torch.Size([2636, 512]) matrix
Extracting features complete in 0m 44s
Computing distance matrix
Computing CMC and mAP
Results ----------
top1:25.8% top5:40.9% top10:48.7% mAP:23.5%
------------------
